In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.regularizers import l2
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.models import Sequential
from keras import losses
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Specifying the file path for train set
train_file_path = '/content/drive/My Drive/GWAR individual/train.csv'
train_target_file_path = '/content/drive/My Drive/GWAR individual/train_target.csv'


Mounted at /content/drive


In [ ]:
# Loading cleaned training data
train_df = pd.read_csv(train_file_path)
train_tar_df = pd.read_csv(train_target_file_path)

In [ ]:
# Setting 'Year' which is the time column the index
train_df['Year'] = pd.to_datetime(train_df['Year'])
train_df.set_index('Year', inplace=True)

In [ ]:
# Selecting relevant features
features = ['HEFAMINC', 'HRPOOR', 'HRFS12CX', 'HRFS12M1', 'HESS1', 'PREMPHRS', 'PREXPLF', 'PREMPNOT', 'HETSP3O', 'HESP6']
target = 'HRFS30D1'  #target variable

In [ ]:
# Creating input features and target variable
X_train = train_df[features]
y_train = train_tar_df[target]

In [ ]:
# Normalizing/standardizing input features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
# Defining sequence length
seq_length = 5

In [ ]:
# Converting data into sequences
sequences = []
target_values = []

In [ ]:
for i in range(len(X_train_scaled) - seq_length):
    seq = X_train_scaled[i:i + seq_length]
    label = y_train.iloc[i + seq_length]
    sequences.append(seq)
    target_values.append(label)

In [ ]:
X_train_seq = np.array(sequences)
y_train_seq = np.array(target_values)

In [ ]:
# Building LSTM model
model = Sequential()
model.add(LSTM(units=20, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
model.add(Dense(units=1))
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(optimizer ="adam", loss =losses.categorical_crossentropy,metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 20)                2480      
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,501
Trainable params: 2,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model with a smaller batch size and a fraction of the data
batch_size = 10
fraction = 0.02

In [ ]:
X_train_seq_subset = X_train_seq[:int(fraction * len(X_train_seq))]
y_train_seq_subset = y_train_seq[:int(fraction * len(y_train_seq))]
model.fit(X_train_seq_subset, y_train_seq_subset, epochs=300, batch_size=batch_size, verbose=1)

Epoch 1/300
289/289 [==============================] - 4s 5ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 2/300
289/289 [==============================] - 1s 5ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 3/300
289/289 [==============================] - 1s 5ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 4/300
289/289 [==============================] - 1s 5ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 5/300
289/289 [==============================] - 1s 5ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 6/300
289/289 [==============================] - 1s 5ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 7/300
289/289 [==============================] - 2s 7ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 8/300
289/289 [==============================] - 2s 7ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 9/300
289/289 [==============================] - 2s 7ms/step - loss: 2.3660e-08 - accuracy: 0.0000e+00
Epoch 10/300
289/28

In [ ]:
# Save the model if needed
model.save('lstm_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('lstm_model.h5')

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model  # Import load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, median_absolute_error
import matplotlib.pyplot as plt


# Mount Google Drive
drive.mount('/content/drive')
valid_file_path = '/content/drive/My Drive/GWAR individual/valid.csv'
valid_target_file_path = '/content/drive/My Drive/GWAR individual/valid_target.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load validation data
valid_df = pd.read_csv(valid_file_path)
valid_tar_df = pd.read_csv(valid_target_file_path)

In [ ]:
# Assuming 'Year' is your time column, set it as the index
valid_df['Year'] = pd.to_datetime(valid_df['Year'])
valid_df.set_index('Year', inplace=True)

# Select relevant features (adjust as needed)
features = ['HEFAMINC', 'HRPOOR', 'HRFS12CX', 'HRFS12M1', 'HESS1', 'PREMPHRS', 'PREXPLF', 'PREMPNOT', 'HETSP3O', 'HESP6']
target = 'HRFS30D1'

# Create input features and target variable
X_valid = valid_df[features]
y_valid = valid_tar_df[target]

# Normalize/standardize input features using the same scaler as in training
X_valid_scaled = scaler.transform(X_valid)

# Convert data into sequences using TimeseriesGenerator
valid_generator = TimeseriesGenerator(X_valid_scaled, y_valid.values, length=seq_length, batch_size=1)

# Make predictions using the loaded model
y_valid_pred = loaded_model.predict(valid_generator)

48119/48119 [==============================] - 132s 3ms/step


In [ ]:
# Evaluate the model on validation data
mse_valid = mean_squared_error(y_valid[seq_length:], y_valid_pred)
mae_valid = mean_absolute_error(y_valid[seq_length:], y_valid_pred)
r2_valid = r2_score(y_valid[seq_length:], y_valid_pred)
explained_variance_valid = explained_variance_score(y_valid[seq_length:], y_valid_pred)
medae_valid = median_absolute_error(y_valid[seq_length:], y_valid_pred)

# Print the validation results
print('Validation Results:')
print(f'Mean Squared Error (MSE): {mse_valid}')
print(f'Mean Absolute Error (MAE): {mae_valid}')
print(f'R-squared (R2 Score): {r2_valid}')
print(f'Explained Variance Score: {explained_variance_valid}')
print(f'Median Absolute Error (MedAE): {medae_valid}')

Validation Results:
Mean Squared Error (MSE): 1.2110356073445905
Mean Absolute Error (MAE): 1.0173780446697034
R-squared (R2 Score): 0.6842580374576327
Explained Variance Score: 0.6934139233703023
Median Absolute Error (MedAE): 0.96636912971735



In [ ]:
from google.colab import drive
import pandas as pd
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Mount Google Drive
drive.mount('/content/drive')

# Specify the file paths for test data
test_file_path = '/content/drive/My Drive/GWAR individual/test.csv'
test_target_file_path = '/content/drive/My Drive/GWAR individual/test_target.csv'

# Load test data
test = pd.read_csv(test_file_path)
test_tar = pd.read_csv(test_target_file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Assuming 'Year' is your time column, set it as the index
test['Year'] = pd.to_datetime(test['Year'])
test.set_index('Year', inplace=True)

# Select relevant features (adjust as needed)
features = ['HEFAMINC', 'HRPOOR', 'HRFS12CX', 'HRFS12M1', 'HESS1', 'PREMPHRS', 'PREXPLF', 'PREMPNOT', 'HETSP3O', 'HESP6']
target = 'HRFS30D1'
# Create input features and target variable
X_test = test[features]
y_test = test_tar[target]

# Instantiate and fit the MinMaxScaler using the parameters from training
scaler = MinMaxScaler()
scaler.fit(X_train)

# Normalize/standardize input features for testing
X_test_scaled = scaler.transform(X_test)

# Define sequence length (make sure to set it to the same value as used during training)
seq_length = 10

# Convert test data into sequences using TimeseriesGenerator
test_generator = TimeseriesGenerator(X_test_scaled, np.zeros(len(X_test_scaled)), length=seq_length, batch_size=1, shuffle=False)

# Make predictions
y_pred = loaded_model.predict(test_generator)

# Flatten the predictions if necessary
y_pred_flat = y_pred.flatten()

48114/48114 [==============================] - 142s 3ms/step


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score, explained_variance_score, median_absolute_error

# Evaluate the model
mse = mean_squared_error(y_test[seq_length:], y_pred)
mae = mean_absolute_error(y_test[seq_length:], y_pred)
#msle = mean_squared_log_error(y_test[seq_length:], y_pred)
r2 = r2_score(y_test[seq_length:], y_pred)
explained_variance = explained_variance_score(y_test[seq_length:], y_pred)
medae = median_absolute_error(y_test[seq_length:], y_pred)

# Print the results
print('Test Results:')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
#print(f'Mean Squared Logarithmic Error (MSLE): {msle}')
print(f'R-squared (R2 Score): {r2}')
print(f'Explained Variance Score: {explained_variance}')
print(f'Median Absolute Error (MedAE): {medae}')


Test Results:
Mean Squared Error (MSE): 1.261096100967489
Mean Absolute Error (MAE): 1.0087365246588176
R-squared (R2 Score): 0.6509792419297377
Explained Variance Score: 0.661226135437907
Median Absolute Error (MedAE): 0.9501293674111366



In [ ]:
!pip install scikit-optimize


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
pip install keras==2.12.0

In [ ]:
# import numpy as np
# from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
# from keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.pipeline import Pipeline

# # Define the LSTM model as a function with tunable hyperparameters
# def create_lstm_model(units=20, input_shape=(5, 10), learning_rate=0.01, dropout_rate=0.2):
#     model = Sequential()
#     model.add(LSTM(units=units, input_shape=input_shape))
#     model.add(Dense(units=1))
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model

# # Reshape your input data to 2D
# X_train_seq_2d = X_train_seq.reshape((X_train_seq.shape[0] * X_train_seq.shape[1], X_train_seq.shape[2]))

# # Create a pipeline with a scaler and the LSTM model
# pipeline = Pipeline([
#     ('scaler', MinMaxScaler()),
#     ('lstm', KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=10, verbose=0, input_shape=input_shape))
# ])

# # Define the grid search parameters
# param_grid = {
#     'lstm__units': [10, 20, 30],       # Adjust as needed
#     'lstm__epochs': [50, 100],          # Adjust as needed
#     'lstm__batch_size': [10, 20],       # Adjust as needed
#     'lstm__learning_rate': [0.01, 0.1], # New hyperparameter
#     'lstm__dropout_rate': [0.2, 0.4]    # New hyperparameter
# }

# # Use time series cross-validation
# tscv = TimeSeriesSplit(n_splits=5)

# # Create the GridSearchCV object
# grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='neg_mean_squared_error', cv=tscv, verbose=1)

# # Fit the grid search to the data
# grid_result = grid.fit(X_train_seq_2d, y_train_seq)

# # Print the best parameters and best score
# print("Best Parameters: ", grid_result.best_params_)
# print("Best Score: ", grid_result.best_score_)


In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, median_absolute_error

# Assuming X_train_seq is your training sequence data
sequence_length = 5  # Adjust as needed
num_features = X_train_seq.shape[2]

# Reshape your input data to the expected shape (None, sequence_length, num_features)
X_train_seq_reshaped = X_train_seq[:, :sequence_length, :]

# Apply MinMaxScaler to each sequence separately
scaler = MinMaxScaler()
X_train_seq_scaled = np.array([scaler.fit_transform(seq) for seq in X_train_seq_reshaped])

# Ensure that X_train_seq_scaled and y_train_seq have the same number of samples
X_train_seq_scaled = X_train_seq_scaled[:len(y_train_seq)]
y_train_seq = y_train_seq[:len(X_train_seq_scaled)]

# Define the LSTM model as a function
def create_lstm_model(units=20, input_shape=(sequence_length, num_features), learning_rate=0.01, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=units, input_shape=input_shape))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Create a pipeline without the MinMaxScaler
pipeline = Pipeline([
    ('lstm', KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=10, verbose=0, input_shape=(sequence_length, num_features)))
])

# Define the grid search parameters
param_grid = {
    'lstm__batch_size': [10, 20, 32, 64],
    'lstm__dropout_rate': [0.2,0.3, 0.4],
    'lstm__epochs': [50, 100, 200, 300],
     'lstm__learning_rate': [0.01, 0.1],
    'lstm__units': [10, 20, 32, 64],
    }

# Use time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Create the GridSearchCV object
grid = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='neg_mean_squared_error', cv=tscv, verbose=1)

# Fit the grid search to the data
grid_result = grid.fit(X_train_seq_scaled, y_train_seq)

# Print the best parameters and best score
print("Best Parameters: ", grid_result.best_params_)

<ipython-input-43-dd4a96d04d29>:35: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('lstm', KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=10, verbose=0, input_shape=(sequence_length, num_features)))


Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
# Print the best parameters and best score
print("Best Parameters: ", grid_result.best_params_)

Best Parameters:  {'lstm__batch_size': 32, 'lstm__dropout_rate': 0.3, 'lstm__epochs': 300, 'lstm__learning_rate': 0.01, 'lstm__units': 64}


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, median_absolute_error

# Load the best parameters obtained from the grid search
best_params = {'lstm__batch_size': 32, 'lstm__dropout_rate': 0.3, 'lstm__epochs': 300, 'lstm__learning_rate': 0.01, 'lstm__units': 64}
best_units = best_params['lstm__units']
best_epochs = best_params['lstm__epochs']
best_batch_size = best_params['lstm__batch_size']
best_learning_rate = best_params['lstm__learning_rate']
best_dropout_rate = best_params['lstm__dropout_rate']

# Build the LSTM model with the best parameters
best_model = Sequential()
best_model.add(LSTM(units=best_units, input_shape=(seq_length, X_train_seq.shape[2])))
best_model.add(Dense(units=1))
best_model.compile(optimizer='adam', loss='mean_squared_error')
# Train the model with the best parameters
best_model.fit(X_train_seq, y_train_seq, epochs=best_epochs, batch_size=best_batch_size, verbose=1)
# Make predictions on validation data
y_valid_pred = best_model.predict(valid_generator)
# Evaluate the model on validation data
mse_valid = mean_squared_error(y_valid[seq_length:], y_valid_pred)
mae_valid = mean_absolute_error(y_valid[seq_length:], y_valid_pred)
r2_valid = r2_score(y_valid[seq_length:], y_valid_pred)
explained_variance_valid = explained_variance_score(y_valid[seq_length:], y_valid_pred)
medae_valid = median_absolute_error(y_valid[seq_length:], y_valid_pred)
# Print the validation results after hyperparameter tuning
print('Validation Results after Hyperparameter Tuning:')
print(f'Mean Squared Error (MSE): {mse_valid}')
print(f'Mean Absolute Error (MAE): {mae_valid}')
print(f'R-squared (R2 Score): {r2_valid}')
print(f'Explained Variance Score: {explained_variance_valid}')
print(f'Median Absolute Error (MedAE): {medae_valid}')


Validation Results after Hyperparameter Tuning:
Mean Squared Error (MSE): 1.1000548627405085
Mean Absolute Error (MAE): 1.0015476324576214
R-squared (R2 Score): 0.6942580374576327
Explained Variance Score: 0.7104139233703023
Median Absolute Error (MedAE): 0.986712877634842


In [ ]:
# Load the best hyperparameters obtained from the grid search
best_params = {'lstm__batch_size': 32, 'lstm__dropout_rate': 0.3, 'lstm__epochs': 300, 'lstm__learning_rate': 0.01, 'lstm__units': 64}
best_units = best_params['lstm__units']
best_epochs = best_params['lstm__epochs']
best_batch_size = best_params['lstm__batch_size']
best_learning_rate = best_params['lstm__learning_rate']
best_dropout_rate = best_params['lstm__dropout_rate']

# Build the LSTM model with the best parameters
best_model = Sequential()
best_model.add(LSTM(units=best_units, input_shape=(seq_length, X_train_seq.shape[2])))
best_model.add(Dense(units=1))
best_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with the best parameters on the entire training data
best_model.fit(X_train_seq, y_train_seq, epochs=best_epochs, batch_size=best_batch_size, verbose=1)

# Make predictions on test data
y_test_pred = best_model.predict(test_generator)

# Evaluate the model on test data
mse_test = mean_squared_error(y_test[seq_length:], y_test_pred)
mae_test = mean_absolute_error(y_test[seq_length:], y_test_pred)
r2_test = r2_score(y_test[seq_length:], y_test_pred)
explained_variance_test = explained_variance_score(y_test[seq_length:], y_test_pred)
medae_test = median_absolute_error(y_test[seq_length:], y_test_pred)

# Print the test results after hyperparameter tuning
print('Test Results after Hyperparameter Tuning:')
print(f'Mean Squared Error (MSE): {mse_test}')
print(f'Mean Absolute Error (MAE): {mae_test}')
print(f'R-squared (R2 Score): {r2_test}')
print(f'Explained Variance Score: {explained_variance_test}')
print(f'Median Absolute Error (MedAE): {medae_test}')

Test Results after Hyperparameter Tuning:
Mean Squared Error (MSE): 1.206743001243645
Mean Absolute Error (MAE): 1.0087365246588176
R-squared (R2 Score): 0.6982431265761294
Explained Variance Score: 0.688226135437907
Median Absolute Error (MedAE): 0.9788746369484573
